In [1]:

def clear_memory():
    !sudo sysctl -w vm.drop_caches=3
clear_memory()


Coloque su dedo en el lector de huellas dactilares
La huella no coincide
Coloque su dedo en el lector de huellas dactilares
vm.drop_caches = 3


In [3]:
import pandas as pd
import polars as pl
import ManipulateData
import MetodoLexico
import DataLoader_Polars
from rapidfuzz import fuzz
import jellyfish
import re
import matplotlib.pyplot as plt

In [4]:
data_loader = DataLoader_Polars.DataLoaderPolars("../DataSets_CSVs/DataSets_Parquets/")
df_concesiones = data_loader.load_concesiones()
manipulator = ManipulateData.ManipulateData(df_concesiones, "", "")

In [5]:
'''
#Para buscar cuales pueden ser las metricas y metodos lexicos correctos a utilizar hacemos una extracción de los titulares únicos a los que nos enfrentamos
titulares_unicos = df_concesiones.sort('titular').select('titular').unique()
'''

"\n#Para buscar cuales pueden ser las metricas y metodos lexicos correctos a utilizar hacemos una extracción de los titulares únicos a los que nos enfrentamos\ntitulares_unicos = df_concesiones.sort('titular').select('titular').unique()\n"

In [6]:
df_concesiones = manipulator.date_string_to_date(df_concesiones)

In [7]:
# Para truncar fechas que ya tienen tiempo
df_concesiones = df_concesiones.with_columns(
    pl.col('ultimo_registro').dt.date().alias('ultimo_registro')
)

In [8]:
#n_nombres_titulares_antes_estandarizar = len(df_concesiones.select('titular').unique())
#n_nombres_titulares_antes_estandarizar

In [9]:
'''
#Para un segundo enfoque podemos normalizar los textos de los titulares con diferentes parámetros

df_concesiones = ManipulateData.ManipulateData(df_concesiones).estandarizar_titular('titular',
                                                                                    sin_espacios=False,
                                                                                    sin_stopword=True,
                                                                                    sin_terminos_mercantiles=True,
                                                                                    quitar_tokens_1_letra=True)
'''

"\n#Para un segundo enfoque podemos normalizar los textos de los titulares con diferentes parámetros\n\ndf_concesiones = ManipulateData.ManipulateData(df_concesiones).estandarizar_titular('titular',\n                                                                                    sin_espacios=False,\n                                                                                    sin_stopword=True,\n                                                                                    sin_terminos_mercantiles=True,\n                                                                                    quitar_tokens_1_letra=True)\n"

In [10]:
#n_nombres_titulares_despues_estandarizar = len(df_concesiones.select('titular').unique())
#n_nombres_titulares_despues_estandarizar

In [11]:
# Seleccionar columnas en el orden deseado

df_concesiones = df_concesiones.select([
    'titulo', 'titular', 'ultimo_registro',
    *[col for col in df_concesiones.columns if col not in ['titulo', 'titular', 'ultimo_registro']]
])


In [12]:
# Ordenar el DataFrame
df_concesiones = df_concesiones.sort([
    pl.col('titulo').count().over('titulo'),
    'titular', 
    'ultimo_registro'
], descending=[True, False, False])

In [13]:
# Obtener información de títulos y titulares
titulos_info = df_concesiones.group_by('titulo').agg([
    pl.col('titular').n_unique().alias('N_titulares')
]).sort('N_titulares', descending=True)



In [14]:
volumenes_totales_info = df_concesiones.group_by('titulo').agg([
    pl.col('volumen_de_extraccion_anual_de_aguas_nacionales_que_ampara_el_titulo_en_m3').n_unique().alias('cambios_volumen'),
    pl.col('volumen_de_extraccion_anual_de_aprovechamientos_subterraneos_en_m3').n_unique().alias('cambios_volumen_subterraneo'),
    pl.col('volumen_de_extraccion_anual_de_aprovechamientos_superficiales_en_m3').n_unique().alias('cambios_volumen_superficial')
])

In [15]:
periodos_info = df_concesiones.group_by('titulo').agg([
    pl.col('ultimo_registro').n_unique().alias('cantidad_periodos'),
    pl.col('ultimo_registro').unique().alias('periodos_presentes'),
    pl.col('ultimo_registro').count().alias('N_cambios_ultimo_registro'),
])

In [16]:

# Recrear df_anomalias con los datos
df_anomalias = pl.DataFrame({
    'titulo': titulos_info['titulo'].to_list(),
    'N_titulares': titulos_info['N_titulares'].to_list(),
    'Score': [None] * len(titulos_info),
    'Diferente_titular': [None] * len(titulos_info),
    'Caracteres_distintos': [None]* len(titulos_info),
    'Cambios_en_20_periodos': periodos_info['N_cambios_ultimo_registro'].to_list(), # Cuantas veces aparece el registro en los 20 periodos
    'Periodos': periodos_info['periodos_presentes'].to_list(), # aqui se ponen los periodos en los que hubieron cambios.
    'Cantidad_periodos': periodos_info['cantidad_periodos'].to_list(), #Cuantos periodos hay en total en dicho registro
    'Anomalia_V_Total': volumenes_totales_info['cambios_volumen'].to_list(),
    'Anomalia_V_Subterraneo': volumenes_totales_info['cambios_volumen_subterraneo'].to_list(),
    'Anomalia_V_Superficial': volumenes_totales_info['cambios_volumen_superficial'].to_list()
})

In [17]:
'''
def length_prefilter(a: str, b: str, base_abs: int = 5, rel_long: float = 0.50) -> bool:
    # Semántica: devuelve True si las longitudes son tan distintas
    # que podemos DESCARTAR el par sin seguir a métodos léxicos.
    L1 = len(a or "")
    L2 = len(b or "")
    Lmax = max(L1, L2)
    if Lmax <= 3:
        return False  # no decidas por longitud cuando son ultra cortos

    # Umbral absoluto dinámico (20% de Lmax, al menos base_abs)
    abs_thr = max(base_abs, int(round(0.20 * Lmax)))

    # Umbral relativo: más exigente en cortos
    rel_thr = 0.40 if Lmax < 10 else rel_long  # p.ej. 0.40 para cortos, 0.50 para largos

    gap_abs = abs(L1 - L2)
    gap_rel = gap_abs / Lmax

    return (gap_abs > abs_thr) and (gap_rel > rel_thr)
'''

'\ndef length_prefilter(a: str, b: str, base_abs: int = 5, rel_long: float = 0.50) -> bool:\n    # Semántica: devuelve True si las longitudes son tan distintas\n    # que podemos DESCARTAR el par sin seguir a métodos léxicos.\n    L1 = len(a or "")\n    L2 = len(b or "")\n    Lmax = max(L1, L2)\n    if Lmax <= 3:\n        return False  # no decidas por longitud cuando son ultra cortos\n\n    # Umbral absoluto dinámico (20% de Lmax, al menos base_abs)\n    abs_thr = max(base_abs, int(round(0.20 * Lmax)))\n\n    # Umbral relativo: más exigente en cortos\n    rel_thr = 0.40 if Lmax < 10 else rel_long  # p.ej. 0.40 para cortos, 0.50 para largos\n\n    gap_abs = abs(L1 - L2)\n    gap_rel = gap_abs / Lmax\n\n    return (gap_abs > abs_thr) and (gap_rel > rel_thr)\n'

In [18]:
'''
def score_y_clase(a:str, b:str):
    # Pre-filtrado por longitud
    if length_prefilter(a, b):
        return 0, 0
    else:
        #metricas
        ts = fuzz.token_set_ratio(a, b)
        jw = jellyfish.jaro_winkler_similarity(a, b) * 100
        score = (ts + jw) / 2
        #reglas    
        if ts >= 96 or (ts >= 93 and jw >= 93):
            clase = 1 #iguales
        elif ts < 90 or (ts < 85 and jw < 88):
            clase = 0 #diferentes
        else:
            clase = 2 
    return score, clase
'''

'\ndef score_y_clase(a:str, b:str):\n    # Pre-filtrado por longitud\n    if length_prefilter(a, b):\n        return 0, 0\n    else:\n        #metricas\n        ts = fuzz.token_set_ratio(a, b)\n        jw = jellyfish.jaro_winkler_similarity(a, b) * 100\n        score = (ts + jw) / 2\n        #reglas    \n        if ts >= 96 or (ts >= 93 and jw >= 93):\n            clase = 1 #iguales\n        elif ts < 90 or (ts < 85 and jw < 88):\n            clase = 0 #diferentes\n        else:\n            clase = 2 \n    return score, clase\n'

In [19]:
def score_y_clase_wrapper(a: str, b: str):
    """Wrapper para usar MetodoLexico en lugar de la función score_y_clase"""
    metodo = MetodoLexico.MetodoLexico(a, b)
    return metodo.score_y_clase()


In [20]:
def get_all_metrics(a: str, b: str):
    """Función que retorna todos los valores de una vez para evitar múltiples instanciaciones"""
    metodo = MetodoLexico.MetodoLexico(a, b)
    score, clase, caracteres_conflictivos = metodo.score_y_clase()
    return (score, clase, caracteres_conflictivos)


In [21]:
# Títulos con exactamente 2 titulares únicos
tit2 = (
    df_concesiones
    .select(['titulo','titular'])
    .unique()
    .group_by('titulo')
    .agg(pl.col('titular').unique().alias('titulares'))
    .with_columns(pl.col('titulares').list.len().alias('N_titulares'))
    .filter(pl.col('N_titulares') == 2)
)

# Extrae pares (t1, t2)
tit2_pairs = tit2.with_columns([
    pl.col('titulares').list.get(0).alias('titular1'),
    pl.col('titulares').list.get(1).alias('titular2')
]).select(['titulo','titular1','titular2'])

In [22]:
tit2_scored = (
    tit2_pairs.with_columns([
        pl.struct(['titular1','titular2']).map_elements(
            lambda s: get_all_metrics(s['titular1'], s['titular2']),
            return_dtype=pl.Object  # <- importante si all_metrics es tuple/list
        ).alias('all_metrics')
    ])
    .with_columns([
        pl.col('all_metrics').map_elements(lambda x: x[0]).alias('Score'),
        pl.col('all_metrics').map_elements(lambda x: x[1]).alias('Diferente_titular'),
        pl.col('all_metrics').map_elements(lambda x: x[2]).alias('Caracteres_distintos'),
    ])
    .drop('all_metrics')
)


/tmp/ipykernel_5046/3898457111.py:8: MapWithoutReturnDtypeWarning: 'return_dtype' of function python_udf must be set

A later expression might fail because the output type is not known. Set return_dtype=pl.self_dtype() if the type is unchanged, or set the proper output data type.
  .with_columns([


In [23]:
df_anomalias = (
    df_anomalias.drop(['Score','Diferente_titular', 'Caracteres_distintos'])  # Eliminar las columnas anteriores
    .join(tit2_scored.select(['titulo','Score','Diferente_titular','Caracteres_distintos']), on='titulo', how='left')  # Incluir Caracteres_distintos
    .select([
        'titulo',
        'N_titulares',
        'Score',
        'Diferente_titular',
        'Caracteres_distintos',
        'Cambios_en_20_periodos',
        'Periodos',
        'Cantidad_periodos',
        'Anomalia_V_Total',
        'Anomalia_V_Subterraneo',
        'Anomalia_V_Superficial'
    ])
)

In [24]:

# fechas = df_concesiones.select('ultimo_registro').unique().sort('ultimo_registro')
# titulos_list = df_concesiones.group_by('titulo').agg(pl.count()).sort('count', descending=True)
# nombres_unicos_en_titulares = df_concesiones.sort('titular').select('titular').unique()
# Distribución de títulos por frecuencia
#dist_titulos_list = titulos_list.group_by('count').agg(pl.count().alias('frequency')).sort('count',descending=True)

In [25]:
'''
plt.bar(dist_titulos_list['count'].to_list(), dist_titulos_list['frequency'].to_list())
plt.yscale('log')
plt.show()
'''

"\nplt.bar(dist_titulos_list['count'].to_list(), dist_titulos_list['frequency'].to_list())\nplt.yscale('log')\nplt.show()\n"

---
---

In [26]:
df_subterraneas = data_loader.load_subterraneas()

In [27]:
df_superficiales = data_loader.load_superficiales()

### Primero comparamos el numero de titulos de cada dataset y vemos que disntan entre si

In [28]:
print("\nTÍTULOS ÚNICOS EN CADA DATAFRAME")
titulos_concesiones = set(df_concesiones.select('titulo').unique().to_series().to_list())
titulos_subterraneas = set(df_subterraneas.select('titulo').unique().to_series().to_list())
titulos_superficiales = set(df_superficiales.select('titulo').unique().to_series().to_list())

print(f"Títulos únicos en df_concesiones: {len(titulos_concesiones)}")
print(f"Títulos únicos en df_subterraneas: {len(titulos_subterraneas)}")
print(f"Títulos únicos en df_superficiales: {len(titulos_superficiales)}")

# Intersecciones
titulos_en_subterraneas_y_concesiones = titulos_concesiones.intersection(titulos_subterraneas)
titulos_en_superficiales_y_concesiones = titulos_concesiones.intersection(titulos_superficiales)
titulos_en_todos = titulos_concesiones.intersection(titulos_subterraneas).intersection(titulos_superficiales)

print(f"\nTítulos que aparecen en concesiones Y subterráneas: {len(titulos_en_subterraneas_y_concesiones)}")
print(f"Títulos que aparecen en concesiones Y superficiales: {len(titulos_en_superficiales_y_concesiones)}")
print(f"Títulos que aparecen en los TRES dataframes: {len(titulos_en_todos)}")


TÍTULOS ÚNICOS EN CADA DATAFRAME
Títulos únicos en df_concesiones: 592469
Títulos únicos en df_subterraneas: 356989
Títulos únicos en df_superficiales: 130314

Títulos que aparecen en concesiones Y subterráneas: 356988
Títulos que aparecen en concesiones Y superficiales: 130314
Títulos que aparecen en los TRES dataframes: 909


In [29]:
# Procesar fechas en los dataframes subterráneas y superficiales (similar a concesiones)
df_subterraneas = manipulator.date_string_to_date(df_subterraneas)
df_superficiales = manipulator.date_string_to_date(df_superficiales)

# Truncar fechas para consistencia
df_subterraneas = df_subterraneas.with_columns(
    pl.col('ultimo_registro').dt.date().alias('ultimo_registro')
)
df_superficiales = df_superficiales.with_columns(
    pl.col('ultimo_registro').dt.date().alias('ultimo_registro')
)

In [30]:
# ANÁLISIS DE COMPARACIÓN DE APROVECHAMIENTOS Y VOLÚMENES POR TÍTULO

def comparar_aprovechamientos_y_volumenes(titulos_concesiones,titulos_subterraneas,titulos_superficiales):
    """
    Compara los aprovechamientos y volúmenes entre df_concesiones, df_subterraneas y df_superficiales
    por título a lo largo de los períodos.
    """
    
    # Identificar columnas de volumen en cada dataframe
    vol_cols_concesiones = [col for col in df_concesiones.columns if 'volumen' in col.lower()]
    vol_cols_subterraneas = [col for col in df_subterraneas.columns if 'volumen' in col.lower()]
    vol_cols_superficiales = [col for col in df_superficiales.columns if 'volumen' in col.lower()]

    print("COLUMNAS RELACIONADAS CON EL VOLUMEN")
    print("df_concesiones:")
    for col in vol_cols_concesiones:
        print(f"  - {col}")
    print("\ndf_subterraneas:")
    for col in vol_cols_subterraneas:
        print(f"  - {col}")
    print("\ndf_superficiales:")
    for col in vol_cols_superficiales:
        print(f"  - {col}")
    
    
    # Títulos para comparables
    titulos_sub_vs_conc = titulos_concesiones.intersection(titulos_subterraneas)
    titulos_sup_vs_conc = titulos_concesiones.intersection(titulos_superficiales)
    
    print(f"\nCantidad de titulos compalabres")
    print(f"Títulos en concesiones Y subterráneas: {len(titulos_sub_vs_conc)}")
    print(f"Títulos en concesiones Y superficiales: {len(titulos_sup_vs_conc)}")
    
    return titulos_sub_vs_conc, titulos_sup_vs_conc, vol_cols_concesiones, vol_cols_subterraneas, vol_cols_superficiales

# Ejecutar análisis inicial


In [31]:
titulos_sub_vs_conc, titulos_sup_vs_conc, vol_cols_concesiones, vol_cols_subterraneas, vol_cols_superficiales = comparar_aprovechamientos_y_volumenes(titulos_concesiones,titulos_subterraneas,titulos_superficiales)

COLUMNAS RELACIONADAS CON EL VOLUMEN
df_concesiones:
  - volumen_de_extraccion_anual_de_aguas_nacionales_que_ampara_el_titulo_en_m3
  - volumen_de_extraccion_anual_de_aprovechamientos_superficiales_en_m3
  - volumen_de_extraccion_anual_de_aprovechamientos_subterraneos_en_m3
  - volumen_de_descarga_diario_en_m3

df_subterraneas:
  - volumen_anual_en_m3

df_superficiales:
  - volumen_anual_en_m3

Cantidad de titulos compalabres
Títulos en concesiones Y subterráneas: 356988
Títulos en concesiones Y superficiales: 130314


In [32]:
# ANÁLISIS DETALLADO DE INCONSISTENCIAS

def analizar_inconsistencias_subterraneas():
    """
    Compara df_concesiones vs df_subterraneas para el mismo título y período
    """
    print("ANÁLISIS DE INCONSISTENCIAS: CONCESIONES vs SUBTERRÁNEAS (MISMO  TITULO Y PERIODO)")
    
    # Preparar datos para comparación
    concesiones_sub = df_concesiones.filter(
        pl.col('titulo').is_in(list(titulos_sub_vs_conc)) #Filtramos para los titulos comparables
    ).select([
        'titulo', 
        'ultimo_registro', 
        'volumen_de_extraccion_anual_de_aprovechamientos_subterraneos_en_m3'
    ]).rename({
        'volumen_de_extraccion_anual_de_aprovechamientos_subterraneos_en_m3': 'vol_subterraneo_concesiones_anual'
    })
    
    subterraneas_comp = df_subterraneas.select([
        'titulo', 
        'ultimo_registro', 
        'volumen_anual_en_m3'
    ]).rename({
        'volumen_anual_en_m3': 'vol_subterraneo_directo_anual'
    })
    
    # Join para comparar
    comparacion_vol_subterranea = concesiones_sub.join(
        subterraneas_comp, 
        on=['titulo', 'ultimo_registro'],  #join solo en estos campos  que coincidan 
        how='inner'
    ).with_columns([
        # Convertir a numérico y manejar nulos
        pl.when(pl.col('vol_subterraneo_concesiones_anual').is_null())
        .then(0.0)
        .otherwise(pl.col('vol_subterraneo_concesiones_anual').cast(pl.Float64, strict=False))
        .alias('vol_conc_num'), #Se crea esta columna para el volumen subterráneo de _concesiones

        pl.when(pl.col('vol_subterraneo_directo_anual').is_null())
        .then(0.0)
        .otherwise(pl.col('vol_subterraneo_directo_anual').cast(pl.Float64, strict=False))
        .alias('vol_sub_num') #Se crea esta columna para el volumen subterráneo de reportados
    ]).with_columns([
        # Calcular diferencias 
        (pl.col('vol_conc_num') - pl.col('vol_sub_num')).alias('diferencia_volumen'),
        pl.when(pl.col('vol_conc_num') == pl.col('vol_sub_num'))
        .then(True)
        .otherwise(False)
        .alias('volumenes_coinciden')
    ])
    #La función height de polars te da el número de filas 
    total_registros = comparacion_vol_subterranea.height
    registros_coinciden = comparacion_vol_subterranea.filter(pl.col('volumenes_coinciden')).height
    registros_difieren = total_registros - registros_coinciden
    
    print(f"Total de registros comparables: {total_registros:,}")
    print(f"Registros con volúmenes coincidentes: {registros_coinciden:,} ({registros_coinciden/total_registros*100:.2f}%)")
    print(f"Registros con volúmenes diferentes: {registros_difieren:,} ({registros_difieren/total_registros*100:.2f}%)")
    
    return comparacion_vol_subterranea

In [33]:
def analizar_inconsistencias_superficiales():
    """
    Compara df_concesiones vs df_superficiales para el mismo título y período
    """
    print("\nANÁLISIS DE INCONSISTENCIAS: CONCESIONES vs SUPERFICIALES(MISMO  TITULO Y PERIODO)")
    
    # Preparar datos para comparación
    concesiones_sup = df_concesiones.filter(
        pl.col('titulo').is_in(list(titulos_sup_vs_conc))
    ).select([
        'titulo', 
        'ultimo_registro', 
        'volumen_de_extraccion_anual_de_aprovechamientos_superficiales_en_m3'
    ]).rename({
        'volumen_de_extraccion_anual_de_aprovechamientos_superficiales_en_m3': 'vol_superficial_concesiones'
    })
    
    superficiales_comp = df_superficiales.select([
        'titulo', 
        'ultimo_registro', 
        'volumen_anual_en_m3'
    ]).rename({
        'volumen_anual_en_m3': 'vol_superficial_directo'
    })
    
    # Join para comparar
    comparacion_vol_superficial = concesiones_sup.join(
        superficiales_comp, 
        on=['titulo', 'ultimo_registro'], 
        how='inner'
    ).with_columns([
        # Convertir a numérico y manejar nulos
        pl.when(pl.col('vol_superficial_concesiones').is_null())
        .then(0.0)
        .otherwise(pl.col('vol_superficial_concesiones').cast(pl.Float64, strict=False))
        .alias('vol_conc_num'),
        
        pl.when(pl.col('vol_superficial_directo').is_null())
        .then(0.0)
        .otherwise(pl.col('vol_superficial_directo').cast(pl.Float64, strict=False))
        .alias('vol_sup_num')
    ]).with_columns([
        # Calcular diferencias
        (pl.col('vol_conc_num') - pl.col('vol_sup_num')).alias('diferencia_volumen'),
        pl.when(pl.col('vol_conc_num') == pl.col('vol_sup_num'))
        .then(True)
        .otherwise(False)
        .alias('volumenes_coinciden')
    ])
    
    total_registros = comparacion_vol_superficial.height
    registros_coinciden = comparacion_vol_superficial.filter(pl.col('volumenes_coinciden')).height
    registros_difieren = total_registros - registros_coinciden
    
    print(f"Total de registros comparables: {total_registros:,}")
    print(f"Registros con volúmenes coincidentes: {registros_coinciden:,} ({registros_coinciden/total_registros*100:.2f}%)")
    print(f"Registros con volúmenes diferentes: {registros_difieren:,} ({registros_difieren/total_registros*100:.2f}%)")
    
    return comparacion_vol_superficial



In [34]:
# Ejecutar análisis
comp_subterraneas = analizar_inconsistencias_subterraneas()


ANÁLISIS DE INCONSISTENCIAS: CONCESIONES vs SUBTERRÁNEAS (MISMO  TITULO Y PERIODO)
Total de registros comparables: 7,746,822
Registros con volúmenes coincidentes: 5,540,380 (71.52%)
Registros con volúmenes diferentes: 2,206,442 (28.48%)


In [35]:
comp_superficiales = analizar_inconsistencias_superficiales()


ANÁLISIS DE INCONSISTENCIAS: CONCESIONES vs SUPERFICIALES(MISMO  TITULO Y PERIODO)
Total de registros comparables: 2,953,039
Registros con volúmenes coincidentes: 2,361,373 (79.96%)
Registros con volúmenes diferentes: 591,666 (20.04%)


In [36]:
# Crear métricas de discrepancias binarias para subterráneas
discrepancias_sub_binarias = comp_subterraneas.group_by('titulo').agg([
    # Si hay al menos una discrepancia, marcar como 1, sino 0
    pl.when(pl.col('volumenes_coinciden').sum() < pl.col('volumenes_coinciden').count())
    .then(1)
    .otherwise(0)
    .alias('Discrepancia_V_Subterraneo')
])

# Crear métricas de discrepancias binarias para superficiales
discrepancias_sup_binarias = comp_superficiales.group_by('titulo').agg([
    # Si hay al menos una discrepancia, marcar como 1, sino 0
    pl.when(pl.col('volumenes_coinciden').sum() < pl.col('volumenes_coinciden').count())
    .then(1)
    .otherwise(0)
    .alias('Discrepancia_V_Superficial')
])






In [37]:
# Agregar las columnas al df_anomalias
df_anomalias = df_anomalias.join(
    discrepancias_sub_binarias, 
    on='titulo', 
    how='left'
).join(
    discrepancias_sup_binarias, 
    on='titulo', 
    how='left'
)

In [39]:
# ANÁLISIS ESTADÍSTICO DE LAS DISCREPANCIAS (CORREGIDO)

def analizar_discrepancias_estadisticas_v2(comparacion_df, tipo_agua):
    """
    Analiza estadísticamente las discrepancias encontradas - versión corregida
    """
    print(f"\nESTADÍSTICAS DE DISCREPANCIAS - {tipo_agua.upper()}")
    
    # Filtrar solo registros con diferencias
    discrepancias = comparacion_df.filter(~pl.col('volumenes_coinciden'))
    
    if discrepancias.height == 0:
        print("No hay discrepancias para analizar")
        return None, None, None
    
    print(f"Total de discrepancias: {discrepancias.height:,}")
    
    # Estadísticas básicas de diferencias
    stats = discrepancias.select([
        pl.col('diferencia_volumen').abs().alias('diff_abs')
    ]).select([
        pl.col('diff_abs').min().alias('min_diff'),
        pl.col('diff_abs').max().alias('max_diff'),
        pl.col('diff_abs').mean().alias('mean_diff'),
        pl.col('diff_abs').median().alias('median_diff'),
        pl.col('diff_abs').std().alias('std_diff'),
        pl.col('diff_abs').quantile(0.25).alias('q25_diff'),
        pl.col('diff_abs').quantile(0.75).alias('q75_diff'),
        pl.col('diff_abs').quantile(0.95).alias('q95_diff'),
        pl.col('diff_abs').quantile(0.99).alias('q99_diff')
    ])
    
    print("\nEstadísticas de diferencias absolutas de volumen:")
    for col in stats.columns:
        val = stats.select(col).item()
        if val is not None:
            print(f"  {col}: {val:,.2f} m³")
    
    # Casos más extremos
    print(f"\nCasos con mayores discrepancias:")
    casos_extremos = discrepancias.select([
        'titulo', 'ultimo_registro', 'vol_conc_num', 'vol_sub_num' if 'vol_sub_num' in discrepancias.columns else 'vol_sup_num',
        'diferencia_volumen'
    ]).with_columns([
        pl.col('diferencia_volumen').abs().alias('diff_abs')
    ]).sort('diff_abs', descending=True).head(10)
    
    print(casos_extremos)
    
    return discrepancias, stats, casos_extremos





In [40]:
# Analizar discrepancias para subterráneas
print("Analizando discrepancias en aguas subterráneas...")
disc_sub, stats_sub, casos_sub = analizar_discrepancias_estadisticas_v2(comp_subterraneas, "subterráneas")

print("\n" + "="*80)

Analizando discrepancias en aguas subterráneas...

ESTADÍSTICAS DE DISCREPANCIAS - SUBTERRÁNEAS
Total de discrepancias: 2,206,442

Estadísticas de diferencias absolutas de volumen:
  min_diff: 0.01 m³
  max_diff: 780,484,464.00 m³
  mean_diff: 8,873,835.66 m³
  median_diff: 29,483.14 m³
  std_diff: 69,936,292.08 m³
  q25_diff: 10,000.00 m³
  q75_diff: 150,000.00 m³
  q95_diff: 8,293,968.00 m³
  q99_diff: 154,009,650.92 m³

Casos con mayores discrepancias:
shape: (10, 6)
┌──────────────────┬─────────────────┬──────────────┬─────────────┬─────────────────┬──────────────┐
│ titulo           ┆ ultimo_registro ┆ vol_conc_num ┆ vol_sub_num ┆ diferencia_volu ┆ diff_abs     │
│ ---              ┆ ---             ┆ ---          ┆ ---         ┆ men             ┆ ---          │
│ str              ┆ date            ┆ f64          ┆ f64         ┆ ---             ┆ f64          │
│                  ┆                 ┆              ┆             ┆ f64             ┆              │
╞══════════════════╪

In [41]:
# Analizar discrepancias para superficiales  
print("Analizando discrepancias en aguas superficiales...")
disc_sup, stats_sup, casos_sup = analizar_discrepancias_estadisticas_v2(comp_superficiales, "superficiales")

Analizando discrepancias en aguas superficiales...

ESTADÍSTICAS DE DISCREPANCIAS - SUPERFICIALES
Total de discrepancias: 591,666

Estadísticas de diferencias absolutas de volumen:
  min_diff: 0.01 m³
  max_diff: 3,393,588,595.00 m³
  mean_diff: 1,362,054.80 m³
  median_diff: 83,603.72 m³
  std_diff: 33,239,709.54 m³
  q25_diff: 15,552.00 m³
  q75_diff: 369,000.00 m³
  q95_diff: 1,521,450.00 m³
  q99_diff: 13,931,912.60 m³

Casos con mayores discrepancias:
shape: (10, 6)
┌───────────────────┬─────────────────┬──────────────┬──────────────┬───────────────────┬──────────┐
│ titulo            ┆ ultimo_registro ┆ vol_conc_num ┆ vol_sup_num  ┆ diferencia_volume ┆ diff_abs │
│ ---               ┆ ---             ┆ ---          ┆ ---          ┆ n                 ┆ ---      │
│ str               ┆ date            ┆ f64          ┆ f64          ┆ ---               ┆ f64      │
│                   ┆                 ┆              ┆              ┆ f64               ┆          │
╞══════════════════

In [42]:
#ANÁLISIS POR TÍTULO - IDENTIFICAR PATRONES DE INCONSISTENCIAS

def analizar_inconsistencias_por_titulo(comp_df, disc_df, tipo_agua):
    """
    Analiza las inconsistencias agrupadas por título para identificar patrones
    """
    print(f"\nANÁLISIS POR TÍTULO - {tipo_agua.upper()}")
    
    # Análisis por título
    titulos_problematicos = disc_df.group_by('titulo').agg([
        pl.count().alias('num_discrepancias'),
        pl.col('diferencia_volumen').abs().mean().alias('discrepancia_promedio'),
        pl.col('diferencia_volumen').abs().max().alias('discrepancia_maxima'),
        pl.col('ultimo_registro').n_unique().alias('periodos_con_discrepancias')
    ]).sort('discrepancia_promedio', descending=True)
    
    print(f"Títulos con discrepancias: {titulos_problematicos.height:,}")
    print(f"Top 10 títulos con mayores discrepancias promedio:")
    print(titulos_problematicos.head(10))
    
    # Análisis temporal
    print(f"\nANÁLISIS TEMPORAL - {tipo_agua.upper()}")
    discrepancias_temporales = disc_df.group_by('ultimo_registro').agg([
        pl.count().alias('num_discrepancias'),
        pl.col('diferencia_volumen').abs().mean().alias('discrepancia_promedio'),
        pl.col('titulo').n_unique().alias('titulos_afectados')
    ]).sort('ultimo_registro')
    
    print("Discrepancias por período (primeros 10):")
    print(discrepancias_temporales.head(10))
    
    # Resumen de consistencia por título
    total_comp = comp_df.group_by('titulo').agg([
        pl.count().alias('total_registros'),
        pl.col('volumenes_coinciden').sum().alias('registros_coincidentes')
    ]).with_columns([
        (pl.col('registros_coincidentes') / pl.col('total_registros') * 100).alias('porcentaje_consistencia')
    ]).sort('porcentaje_consistencia')
    
    print(f"\nTítulos con menor consistencia (menores porcentajes de coincidencia):")
    print(total_comp.head(10))
    
    return titulos_problematicos, discrepancias_temporales, total_comp

# Ejecutar análisis por título
if disc_sub is not None:
    tit_prob_sub, disc_temp_sub, consist_sub = analizar_inconsistencias_por_titulo(comp_subterraneas, disc_sub, "subterráneas")

print("\n" + "="*80)

if disc_sup is not None:
    tit_prob_sup, disc_temp_sup, consist_sup = analizar_inconsistencias_por_titulo(comp_superficiales, disc_sup, "superficiales")


ANÁLISIS POR TÍTULO - SUBTERRÁNEAS


Títulos con discrepancias: 27,530
Top 10 títulos con mayores discrepancias promedio:
shape: (10, 5)
┌───────────────────┬───────────────────┬───────────────────┬───────────────────┬──────────────────┐
│ titulo            ┆ num_discrepancias ┆ discrepancia_prom ┆ discrepancia_maxi ┆ periodos_con_dis │
│ ---               ┆ ---               ┆ edio              ┆ ma                ┆ crepancias       │
│ str               ┆ u32               ┆ ---               ┆ ---               ┆ ---              │
│                   ┆                   ┆ f64               ┆ f64               ┆ u32              │
╞═══════════════════╪═══════════════════╪═══════════════════╪═══════════════════╪══════════════════╡
│ 13DFE100309/26HMG ┆ 17160             ┆ 7.7961e8          ┆ 7.80484464e8      ┆ 20               │
│ C18               ┆                   ┆                   ┆                   ┆                  │
│ 13MEX100314/26HMS ┆ 80                ┆ 1.5447e8          ┆ 1.9001e8          ┆ 20        

/tmp/ipykernel_5046/1528320553.py:11: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('num_discrepancias'),
/tmp/ipykernel_5046/1528320553.py:24: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('num_discrepancias'),
/tmp/ipykernel_5046/1528320553.py:34: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('total_registros'),



Títulos con menor consistencia (menores porcentajes de coincidencia):
shape: (10, 4)
┌──────────────────────┬─────────────────┬────────────────────────┬─────────────────────────┐
│ titulo               ┆ total_registros ┆ registros_coincidentes ┆ porcentaje_consistencia │
│ ---                  ┆ ---             ┆ ---                    ┆ ---                     │
│ str                  ┆ u32             ┆ u32                    ┆ f64                     │
╞══════════════════════╪═════════════════╪════════════════════════╪═════════════════════════╡
│ 04TLX150157/18HMDL18 ┆ 40              ┆ 0                      ┆ 0.0                     │
│ 06COA108159/24ASGE99 ┆ 40              ┆ 0                      ┆ 0.0                     │
│ 858893               ┆ 8               ┆ 0                      ┆ 0.0                     │
│ 05GRO115745/19ESOC07 ┆ 60              ┆ 0                      ┆ 0.0                     │
│ 13HGO100055/26IMDL08 ┆ 300             ┆ 0                      ┆ 

In [43]:
# RESUMEN CONSOLIDADO Y CONCLUSIONES

print("="*100)
print("RESUMEN CONSOLIDADO DEL ANÁLISIS")
print("="*100)

print("\n🔍 HALLAZGOS PRINCIPALES:")

print(f"\n1. COBERTURA DE DATOS:")
print(f"   • Títulos únicos en df_concesiones: {len(titulos_concesiones):,}")
print(f"   • Títulos únicos en df_subterraneas: {len(titulos_subterraneas):,}")
print(f"   • Títulos únicos en df_superficiales: {len(titulos_superficiales):,}")
print(f"   • Títulos comparables (concesiones ∩ subterráneas): {len(titulos_sub_vs_conc):,}")
print(f"   • Títulos comparables (concesiones ∩ superficiales): {len(titulos_sup_vs_conc):,}")

print(f"\n2. CONSISTENCIA DE VOLÚMENES:")
total_sub = comp_subterraneas.height
coincidentes_sub = comp_subterraneas.filter(pl.col('volumenes_coinciden')).height
total_sup = comp_superficiales.height
coincidentes_sup = comp_superficiales.filter(pl.col('volumenes_coinciden')).height

print(f"   🌊 AGUAS SUBTERRÁNEAS:")
print(f"      • Registros comparables: {total_sub:,}")
print(f"      • Coincidencias: {coincidentes_sub:,} ({coincidentes_sub/total_sub*100:.2f}%)")
print(f"      • Discrepancias: {total_sub-coincidentes_sub:,} ({(total_sub-coincidentes_sub)/total_sub*100:.2f}%)")

print(f"   🏞️ AGUAS SUPERFICIALES:")
print(f"      • Registros comparables: {total_sup:,}")
print(f"      • Coincidencias: {coincidentes_sup:,} ({coincidentes_sup/total_sup*100:.2f}%)")
print(f"      • Discrepancias: {total_sup-coincidentes_sup:,} ({(total_sup-coincidentes_sup)/total_sup*100:.2f}%)")

print(f"\n3. SEVERIDAD DE DISCREPANCIAS:")
if stats_sub is not None:
    median_sub = stats_sub.select('median_diff').item()
    mean_sub = stats_sub.select('mean_diff').item()
    print(f"   🌊 SUBTERRÁNEAS: Mediana = {median_sub:,.2f} m³, Promedio = {mean_sub:,.2f} m³")

if stats_sup is not None:
    median_sup = stats_sup.select('median_diff').item()
    mean_sup = stats_sup.select('mean_diff').item()
    print(f"   🏞️ SUPERFICIALES: Mediana = {median_sup:,.2f} m³, Promedio = {mean_sup:,.2f} m³")

print(f"\n📊 INTERPRETACIÓN:")
print(f"   • Las aguas superficiales muestran MAYOR consistencia ({coincidentes_sup/total_sup*100:.1f}%)")
print(f"   • Las aguas subterráneas tienen MÁS discrepancias ({(total_sub-coincidentes_sub)/total_sub*100:.1f}%)")
print(f"   • Esto sugiere posibles problemas de calidad de datos o diferencias en metodologías")
print(f"   • Las discrepancias son significativas en magnitud (promedio en millones de m³)")

print(f"\n⚠️ RECOMENDACIONES:")
print(f"   1. Investigar los títulos con mayor inconsistencia identificados")
print(f"   2. Revisar la metodología de captura de datos entre sistemas")
print(f"   3. Implementar validaciones cruzadas automáticas")
print(f"   4. Priorizar la limpieza de datos en registros subterráneos")
print(f"   5. Establecer alertas para discrepancias > umbral definido")

print("="*100)

RESUMEN CONSOLIDADO DEL ANÁLISIS

🔍 HALLAZGOS PRINCIPALES:

1. COBERTURA DE DATOS:
   • Títulos únicos en df_concesiones: 592,469
   • Títulos únicos en df_subterraneas: 356,989
   • Títulos únicos en df_superficiales: 130,314
   • Títulos comparables (concesiones ∩ subterráneas): 356,988
   • Títulos comparables (concesiones ∩ superficiales): 130,314

2. CONSISTENCIA DE VOLÚMENES:
   🌊 AGUAS SUBTERRÁNEAS:
      • Registros comparables: 7,746,822
      • Coincidencias: 5,540,380 (71.52%)
      • Discrepancias: 2,206,442 (28.48%)
   🏞️ AGUAS SUPERFICIALES:
      • Registros comparables: 2,953,039
      • Coincidencias: 2,361,373 (79.96%)
      • Discrepancias: 591,666 (20.04%)

3. SEVERIDAD DE DISCREPANCIAS:
   🌊 SUBTERRÁNEAS: Mediana = 29,483.14 m³, Promedio = 8,873,835.66 m³
   🏞️ SUPERFICIALES: Mediana = 83,603.72 m³, Promedio = 1,362,054.80 m³

📊 INTERPRETACIÓN:
   • Las aguas superficiales muestran MAYOR consistencia (80.0%)
   • Las aguas subterráneas tienen MÁS discrepancias (28.

In [44]:
# INTEGRACIÓN DE DISCREPANCIAS AL DF_ANOMALIAS

def crear_metricas_discrepancias():
    """
    Calcula métricas de discrepancias por título para integrar al df_anomalias
    """
    print("CREANDO MÉTRICAS DE DISCREPANCIAS POR TÍTULO")
    
    # === MÉTRICAS PARA AGUAS SUBTERRÁNEAS ===
    metricas_sub = None
    if disc_sub is not None and disc_sub.height > 0:
        metricas_sub = disc_sub.group_by('titulo').agg([
            pl.count().alias('discrepancias_subterraneas_count'),
            pl.col('diferencia_volumen').abs().mean().alias('discrepancia_subterranea_promedio'),
            pl.col('diferencia_volumen').abs().max().alias('discrepancia_subterranea_maxima'),
            pl.col('diferencia_volumen').abs().median().alias('discrepancia_subterranea_mediana'),
            pl.col('ultimo_registro').n_unique().alias('periodos_con_discrepancia_sub')
        ])
        print(f"Métricas subterráneas calculadas para {metricas_sub.height:,} títulos")
    
    # === MÉTRICAS PARA AGUAS SUPERFICIALES ===
    metricas_sup = None
    if disc_sup is not None and disc_sup.height > 0:
        metricas_sup = disc_sup.group_by('titulo').agg([
            pl.count().alias('discrepancias_superficiales_count'),
            pl.col('diferencia_volumen').abs().mean().alias('discrepancia_superficial_promedio'),
            pl.col('diferencia_volumen').abs().max().alias('discrepancia_superficial_maxima'),
            pl.col('diferencia_volumen').abs().median().alias('discrepancia_superficial_mediana'),
            pl.col('ultimo_registro').n_unique().alias('periodos_con_discrepancia_sup')
        ])
        print(f"Métricas superficiales calculadas para {metricas_sup.height:,} títulos")
    
    # === MÉTRICAS DE CONSISTENCIA GENERAL ===
    # Para subterráneas
    consistencia_sub = None
    if 'consist_sub' in globals() and consist_sub is not None:
        consistencia_sub = consist_sub.select([
            'titulo',
            'total_registros',
            'registros_coincidentes', 
            'porcentaje_consistencia'
        ]).rename({
            'total_registros': 'total_registros_sub',
            'registros_coincidentes': 'registros_coincidentes_sub',
            'porcentaje_consistencia': 'porcentaje_consistencia_sub'
        })
        print(f"Consistencia subterráneas calculada para {consistencia_sub.height:,} títulos")
    
    # Para superficiales
    consistencia_sup = None
    if 'consist_sup' in globals() and consist_sup is not None:
        consistencia_sup = consist_sup.select([
            'titulo',
            'total_registros',
            'registros_coincidentes', 
            'porcentaje_consistencia'
        ]).rename({
            'total_registros': 'total_registros_sup',
            'registros_coincidentes': 'registros_coincidentes_sup',
            'porcentaje_consistencia': 'porcentaje_consistencia_sup'
        })
        print(f"Consistencia superficiales calculada para {consistencia_sup.height:,} títulos")
    
    return metricas_sub, metricas_sup, consistencia_sub, consistencia_sup

# Ejecutar cálculo de métricas
metricas_sub, metricas_sup, consistencia_sub, consistencia_sup = crear_metricas_discrepancias()

CREANDO MÉTRICAS DE DISCREPANCIAS POR TÍTULO
Métricas subterráneas calculadas para 27,530 títulos
Métricas superficiales calculadas para 6,678 títulos
Consistencia subterráneas calculada para 356,988 títulos
Consistencia superficiales calculada para 130,314 títulos


/tmp/ipykernel_5046/1805708906.py:13: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('discrepancias_subterraneas_count'),
/tmp/ipykernel_5046/1805708906.py:25: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('discrepancias_superficiales_count'),


In [45]:
# INTEGRAR MÉTRICAS AL DF_ANOMALIAS

def integrar_discrepancias_a_anomalias():
    """
    Integra las métricas de discrepancias al dataframe df_anomalias existente
    """
    print("=== INTEGRANDO DISCREPANCIAS AL DF_ANOMALIAS ===")
    
    # Comenzar con el df_anomalias actual
    df_anomalias_expandido = df_anomalias
    
    # === INTEGRAR MÉTRICAS DE SUBTERRÁNEAS ===
    if metricas_sub is not None:
        df_anomalias_expandido = df_anomalias_expandido.join(
            metricas_sub, 
            on='titulo', 
            how='left'
        )
        print(f"✓ Métricas de discrepancias subterráneas integradas")
    else:
        # Agregar columnas vacías si no hay métricas
        df_anomalias_expandido = df_anomalias_expandido.with_columns([
            pl.lit(None).alias('discrepancias_subterraneas_count'),
            pl.lit(None).alias('discrepancia_subterranea_promedio'),
            pl.lit(None).alias('discrepancia_subterranea_maxima'),
            pl.lit(None).alias('discrepancia_subterranea_mediana'),
            pl.lit(None).alias('periodos_con_discrepancia_sub')
        ])
    
    # === INTEGRAR MÉTRICAS DE SUPERFICIALES ===
    if metricas_sup is not None:
        df_anomalias_expandido = df_anomalias_expandido.join(
            metricas_sup, 
            on='titulo', 
            how='left'
        )
        print(f"✓ Métricas de discrepancias superficiales integradas")
    else:
        # Agregar columnas vacías si no hay métricas
        df_anomalias_expandido = df_anomalias_expandido.with_columns([
            pl.lit(None).alias('discrepancias_superficiales_count'),
            pl.lit(None).alias('discrepancia_superficial_promedio'),
            pl.lit(None).alias('discrepancia_superficial_maxima'),
            pl.lit(None).alias('discrepancia_superficial_mediana'),
            pl.lit(None).alias('periodos_con_discrepancia_sup')
        ])
    
    # === INTEGRAR MÉTRICAS DE CONSISTENCIA SUBTERRÁNEAS ===
    if consistencia_sub is not None:
        df_anomalias_expandido = df_anomalias_expandido.join(
            consistencia_sub, 
            on='titulo', 
            how='left'
        )
        print(f"✓ Métricas de consistencia subterráneas integradas")
    else:
        df_anomalias_expandido = df_anomalias_expandido.with_columns([
            pl.lit(None).alias('total_registros_sub'),
            pl.lit(None).alias('registros_coincidentes_sub'),
            pl.lit(None).alias('porcentaje_consistencia_sub')
        ])
    
    # === INTEGRAR MÉTRICAS DE CONSISTENCIA SUPERFICIALES ===
    if consistencia_sup is not None:
        df_anomalias_expandido = df_anomalias_expandido.join(
            consistencia_sup, 
            on='titulo', 
            how='left'
        )
        print(f"✓ Métricas de consistencia superficiales integradas")
    else:
        df_anomalias_expandido = df_anomalias_expandido.with_columns([
            pl.lit(None).alias('total_registros_sup'),
            pl.lit(None).alias('registros_coincidentes_sup'),
            pl.lit(None).alias('porcentaje_consistencia_sup')
        ])
    
    return df_anomalias_expandido

# Ejecutar integración
df_anomalias_completo = integrar_discrepancias_a_anomalias()

print(f"\n=== RESULTADO DE LA INTEGRACIÓN ===")
print(f"Forma original de df_anomalias: {df_anomalias.shape}")
print(f"Forma expandida de df_anomalias_completo: {df_anomalias_completo.shape}")
print(f"\nNuevas columnas agregadas:")
nuevas_columnas = [col for col in df_anomalias_completo.columns if col not in df_anomalias.columns]
for col in nuevas_columnas:
    print(f"  - {col}")

=== INTEGRANDO DISCREPANCIAS AL DF_ANOMALIAS ===
✓ Métricas de discrepancias subterráneas integradas
✓ Métricas de discrepancias superficiales integradas
✓ Métricas de consistencia subterráneas integradas
✓ Métricas de consistencia superficiales integradas

=== RESULTADO DE LA INTEGRACIÓN ===
Forma original de df_anomalias: (592469, 13)
Forma expandida de df_anomalias_completo: (592469, 29)

Nuevas columnas agregadas:
  - discrepancias_subterraneas_count
  - discrepancia_subterranea_promedio
  - discrepancia_subterranea_maxima
  - discrepancia_subterranea_mediana
  - periodos_con_discrepancia_sub
  - discrepancias_superficiales_count
  - discrepancia_superficial_promedio
  - discrepancia_superficial_maxima
  - discrepancia_superficial_mediana
  - periodos_con_discrepancia_sup
  - total_registros_sub
  - registros_coincidentes_sub
  - porcentaje_consistencia_sub
  - total_registros_sup
  - registros_coincidentes_sup
  - porcentaje_consistencia_sup


In [46]:
# ANÁLISIS FINAL Y SCORE DE ANOMALÍA INTEGRADO

def crear_score_anomalia_integrado(df):
    """
    Crea un score de anomalía que integra todas las discrepancias encontradas
    """
    print("=== CREANDO SCORE DE ANOMALÍA INTEGRADO ===")
    
    df_scored = df.with_columns([
        # Score de anomalías de titulares (ya existente)
        pl.when(pl.col('N_titulares') > 1)
        .then(
            pl.when(pl.col('Diferente_titular') == 0)  # Titulares diferentes
            .then(10.0)
            .when(pl.col('Diferente_titular') == 2)    # Titulares dudosos
            .then(5.0)
            .otherwise(0.0)
        )
        .otherwise(0.0)
        .alias('score_titulares'),
        
        # Score de discrepancias subterráneas
        pl.when(pl.col('discrepancias_subterraneas_count').is_not_null())
        .then(
            pl.when(pl.col('porcentaje_consistencia_sub') < 50.0)
            .then(8.0)
            .when(pl.col('porcentaje_consistencia_sub') < 75.0)
            .then(5.0)
            .when(pl.col('porcentaje_consistencia_sub') < 90.0)
            .then(3.0)
            .otherwise(1.0)
        )
        .otherwise(0.0)
        .alias('score_discrepancias_sub'),
        
        # Score de discrepancias superficiales
        pl.when(pl.col('discrepancias_superficiales_count').is_not_null())
        .then(
            pl.when(pl.col('porcentaje_consistencia_sup') < 50.0)
            .then(8.0)
            .when(pl.col('porcentaje_consistencia_sup') < 75.0)
            .then(5.0)
            .when(pl.col('porcentaje_consistencia_sup') < 90.0)
            .then(3.0)
            .otherwise(1.0)
        )
        .otherwise(0.0)
        .alias('score_discrepancias_sup'),
        
        # Score de anomalías de volumen (ya existente)
        (
            pl.when(pl.col('Anomalia_V_Total') > 1).then(2.0).otherwise(0.0) +
            pl.when(pl.col('Anomalia_V_Subterraneo') > 1).then(2.0).otherwise(0.0) +
            pl.when(pl.col('Anomalia_V_Superficial') > 1).then(2.0).otherwise(0.0)
        ).alias('score_volumenes'),
        
        # Score de frecuencia de cambios
        pl.when(pl.col('Cambios_en_20_periodos') > 15)
        .then(3.0)
        .when(pl.col('Cambios_en_20_periodos') > 10)
        .then(2.0)
        .when(pl.col('Cambios_en_20_periodos') > 5)
        .then(1.0)
        .otherwise(0.0)
        .alias('score_frecuencia_cambios')
    ])
    
    # Agregar score total en una operación separada
    df_scored = df_scored.with_columns([
        # Score total de anomalía
        (
            pl.col('score_titulares') + 
            pl.col('score_discrepancias_sub') + 
            pl.col('score_discrepancias_sup') + 
            pl.col('score_volumenes') + 
            pl.col('score_frecuencia_cambios')
        ).alias('score_anomalia_total')
    ])
    
    # Agregar clasificación en otra operación separada
    df_scored = df_scored.with_columns([
        # Clasificación de anomalía
        pl.when(pl.col('score_anomalia_total') >= 15)
        .then(pl.lit('CRÍTICA'))
        .when(pl.col('score_anomalia_total') >= 10)
        .then(pl.lit('ALTA'))
        .when(pl.col('score_anomalia_total') >= 5)
        .then(pl.lit('MEDIA'))
        .when(pl.col('score_anomalia_total') > 0)
        .then(pl.lit('BAJA'))
        .otherwise(pl.lit('SIN_ANOMALIAS'))
        .alias('clasificacion_anomalia')
    ])
    
    return df_scored

# Aplicar scoring
df_anomalias_final = crear_score_anomalia_integrado(df_anomalias_completo)

print(f"\n=== DISTRIBUCIÓN DE ANOMALÍAS ===")
distribucion = df_anomalias_final.group_by('clasificacion_anomalia').agg([
    pl.len().alias('cantidad'),
    pl.col('score_anomalia_total').mean().alias('score_promedio')
]).sort('score_promedio', descending=True)

print(distribucion)

print(f"\n=== TOP 10 TÍTULOS CON MAYOR SCORE DE ANOMALÍA ===")
top_anomalias = df_anomalias_final.sort('score_anomalia_total', descending=True).head(10)
print(top_anomalias.select([
    'titulo', 
    'N_titulares',
    'Diferente_titular',
    'porcentaje_consistencia_sub',
    'porcentaje_consistencia_sup',
    'score_anomalia_total',
    'clasificacion_anomalia'
]))

=== CREANDO SCORE DE ANOMALÍA INTEGRADO ===

=== DISTRIBUCIÓN DE ANOMALÍAS ===
shape: (5, 3)
┌────────────────────────┬──────────┬────────────────┐
│ clasificacion_anomalia ┆ cantidad ┆ score_promedio │
│ ---                    ┆ ---      ┆ ---            │
│ str                    ┆ u32      ┆ f64            │
╞════════════════════════╪══════════╪════════════════╡
│ CRÍTICA                ┆ 756      ┆ 17.238095      │
│ ALTA                   ┆ 30409    ┆ 11.003091      │
│ MEDIA                  ┆ 11041    ┆ 7.81623        │
│ BAJA                   ┆ 517263   ┆ 2.789683       │
│ SIN_ANOMALIAS          ┆ 33000    ┆ 0.0            │
└────────────────────────┴──────────┴────────────────┘

=== TOP 10 TÍTULOS CON MAYOR SCORE DE ANOMALÍA ===
shape: (10, 7)
┌──────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
│ titulo       ┆ N_titulares ┆ Diferente_t ┆ porcentaje_ ┆ porcentaje_ ┆ score_anoma ┆ clasificaci │
│ ---          ┆ ---         ┆ it

In [47]:
# REPORTE FINAL Y EXPORTACIÓN DE RESULTADOS

def generar_reporte_final():
    """
    Genera un reporte final completo del análisis de anomalías
    """
    print("="*100)
    print("                    REPORTE FINAL DE ANÁLISIS DE ANOMALÍAS")
    print("="*100)
    
    print("\nRESUMEN EJECUTIVO:")
    print(f"   • Total de títulos analizados: {df_anomalias_final.height:,}")
    
    # Distribución de anomalías
    distribucion = df_anomalias_final.group_by('clasificacion_anomalia').agg([
        pl.len().alias('cantidad'),
        (pl.len() / df_anomalias_final.height * 100).alias('porcentaje')
    ]).sort('cantidad', descending=True)
    
    print(f"\nDISTRIBUCIÓN DE ANOMALÍAS:")
    for row in distribucion.iter_rows(named=True):
        print(f"   • {row['clasificacion_anomalia']}: {row['cantidad']:,} títulos ({row['porcentaje']:.2f}%)")
    
    # Títulos con anomalías críticas y altas
    criticas = df_anomalias_final.filter(pl.col('clasificacion_anomalia') == 'CRÍTICA').height
    altas = df_anomalias_final.filter(pl.col('clasificacion_anomalia') == 'ALTA').height

    print(f"\n⚠️ TÍTULOS PRIORITARIOS:")
    print(f"   • Anomalías CRÍTICAS: {criticas:,} títulos")
    print(f"   • Anomalías ALTAS: {altas:,} títulos")
    print(f"   • Total prioritarios: {criticas + altas:,} títulos ({(criticas + altas)/df_anomalias_final.height*100:.2f}%)")
    
    # Análisis por componente de anomalía
    print(f"\n🔍 ANÁLISIS POR COMPONENTE:")
    
    # Anomalías de titulares
    titulares_problematicos = df_anomalias_final.filter(pl.col('score_titulares') > 0).height
    print(f"   • Títulos con problemas de titulares: {titulares_problematicos:,}")
    
    # Discrepancias subterráneas
    disc_sub_count = df_anomalias_final.filter(
        pl.col('discrepancias_subterraneas_count').is_not_null() & 
        (pl.col('discrepancias_subterraneas_count') > 0)
    ).height
    print(f"   • Títulos con discrepancias subterráneas: {disc_sub_count:,}")
    
    # Discrepancias superficiales
    disc_sup_count = df_anomalias_final.filter(
        pl.col('discrepancias_superficiales_count').is_not_null() & 
        (pl.col('discrepancias_superficiales_count') > 0)
    ).height
    print(f"   • Títulos con discrepancias superficiales: {disc_sup_count:,}")
    
    # Anomalías de volumen
    vol_anomalias = df_anomalias_final.filter(pl.col('score_volumenes') > 0).height
    print(f"   • Títulos con anomalías de volumen: {vol_anomalias:,}")
    
    print(f"\n📈 ESTADÍSTICAS DE SCORING:")
    stats_scoring = df_anomalias_final.select([
        pl.col('score_anomalia_total').min().alias('score_min'),
        pl.col('score_anomalia_total').max().alias('score_max'),
        pl.col('score_anomalia_total').mean().alias('score_promedio'),
        pl.col('score_anomalia_total').median().alias('score_mediana')
    ])
    
    for row in stats_scoring.iter_rows(named=True):
        print(f"   • Score mínimo: {row['score_min']:.2f}")
        print(f"   • Score máximo: {row['score_max']:.2f}")
        print(f"   • Score promedio: {row['score_promedio']:.2f}")
        print(f"   • Score mediana: {row['score_mediana']:.2f}")
    
    return distribucion

def exportar_resultados():
    """
    Exporta los resultados a archivos CSV para análisis posterior
    """
    print(f"\n💾 EXPORTANDO RESULTADOS...")
    
    # Seleccionar solo columnas numéricas y de texto para exportación
    columnas_exportables = [
        'titulo', 'N_titulares', 'Score', 'Diferente_titular',
        'Cambios_en_20_periodos', 'Cantidad_periodos',
        'Anomalia_V_Total', 'Anomalia_V_Subterraneo', 'Anomalia_V_Superficial',
        'discrepancias_subterraneas_count', 'discrepancia_subterranea_promedio',
        'discrepancia_subterranea_maxima', 'discrepancia_subterranea_mediana',
        'discrepancias_superficiales_count', 'discrepancia_superficial_promedio', 
        'discrepancia_superficial_maxima', 'discrepancia_superficial_mediana',
        'total_registros_sub', 'registros_coincidentes_sub', 'porcentaje_consistencia_sub',
        'total_registros_sup', 'registros_coincidentes_sup', 'porcentaje_consistencia_sup',
        'score_titulares', 'score_discrepancias_sub', 'score_discrepancias_sup',
        'score_volumenes', 'score_frecuencia_cambios', 'score_anomalia_total',
        'clasificacion_anomalia'
    ]
    
    # Crear dataframe para exportación
    df_export = df_anomalias_final.select([col for col in columnas_exportables if col in df_anomalias_final.columns])
    
    # Exportar df_anomalias_final completo
    df_export.write_csv("df_anomalias_completo.csv")
    print(f"   ✓ df_anomalias_completo.csv exportado")
    
    # Exportar solo títulos con anomalías críticas y altas
    prioritarios = df_export.filter(
        pl.col('clasificacion_anomalia').is_in(['CRÍTICA', 'ALTA'])
    ).sort('score_anomalia_total', descending=True)
    
    prioritarios.write_csv("titulos_prioritarios.csv")
    print(f"   ✓ titulos_prioritarios.csv exportado ({prioritarios.height:,} registros)")
    
    # Exportar top 100 títulos con mayor score
    top_100 = df_export.sort('score_anomalia_total', descending=True).head(100)
    top_100.write_csv("top_100_anomalias.csv")
    print(f"   ✓ top_100_anomalias.csv exportado")
    
    # Resumen estadístico
    resumen = df_anomalias_final.group_by('clasificacion_anomalia').agg([
        pl.len().alias('cantidad'),
        pl.col('score_anomalia_total').mean().alias('score_promedio'),
        pl.col('score_anomalia_total').min().alias('score_min'),
        pl.col('score_anomalia_total').max().alias('score_max'),
        pl.col('N_titulares').mean().alias('titulares_promedio'),
        pl.col('discrepancias_subterraneas_count').sum().alias('total_discrepancias_sub'),
        pl.col('discrepancias_superficiales_count').sum().alias('total_discrepancias_sup')
    ])
    
    resumen.write_csv("resumen_anomalias.csv")
    print(f"   ✓ resumen_anomalias.csv exportado")
    
    return prioritarios, top_100

# Ejecutar reporte y exportación
distribucion_final = generar_reporte_final()
prioritarios, top_100 = exportar_resultados()

print(f"\n🎯 RECOMENDACIONES FINALES:")
print(f"   1. Priorizar revisión de {prioritarios.height:,} títulos críticos/altos")
print(f"   2. Implementar validaciones automáticas para discrepancias de volumen")
print(f"   3. Revisar metodología de captura para aguas subterráneas")
print(f"   4. Establecer alertas para títulos con score > 10")
print(f"   5. Auditar títulos con múltiples titulares conflictivos")

print("="*100)

                    REPORTE FINAL DE ANÁLISIS DE ANOMALÍAS

RESUMEN EJECUTIVO:
   • Total de títulos analizados: 592,469

DISTRIBUCIÓN DE ANOMALÍAS:
   • BAJA: 517,263 títulos (87.31%)
   • SIN_ANOMALIAS: 33,000 títulos (5.57%)
   • ALTA: 30,409 títulos (5.13%)
   • MEDIA: 11,041 títulos (1.86%)
   • CRÍTICA: 756 títulos (0.13%)



⚠️ TÍTULOS PRIORITARIOS:
   • Anomalías CRÍTICAS: 756 títulos
   • Anomalías ALTAS: 30,409 títulos
   • Total prioritarios: 31,165 títulos (5.26%)

🔍 ANÁLISIS POR COMPONENTE:
   • Títulos con problemas de titulares: 7,013
   • Títulos con discrepancias subterráneas: 27,530
   • Títulos con discrepancias superficiales: 6,678
   • Títulos con anomalías de volumen: 1,946

📈 ESTADÍSTICAS DE SCORING:
   • Score mínimo: 0.00
   • Score máximo: 23.00
   • Score promedio: 3.17
   • Score mediana: 3.00

💾 EXPORTANDO RESULTADOS...
   ✓ df_anomalias_completo.csv exportado
   ✓ titulos_prioritarios.csv exportado (31,165 registros)
   ✓ top_100_anomalias.csv exportado
   ✓ resumen_anomalias.csv exportado

🎯 RECOMENDACIONES FINALES:
   1. Priorizar revisión de 31,165 títulos críticos/altos
   2. Implementar validaciones automáticas para discrepancias de volumen
   3. Revisar metodología de captura para aguas subterráneas
   4. Establecer alertas para títulos con score > 10
   5. Auditar títulos con

In [48]:
# EJEMPLOS DE TÍTULOS PROBLEMÁTICOS PARA VALIDACIÓN

def mostrar_ejemplos_problematicos():
    """
    Muestra ejemplos específicos de los títulos más problemáticos para validación manual
    """
    print("="*100)
    print("                        EJEMPLOS DE TÍTULOS PROBLEMÁTICOS")
    print("="*100)
    
    # TOP 5 títulos con score más alto
    print("\n🚨 TOP 5 TÍTULOS CON MAYOR SCORE DE ANOMALÍA:")
    top_5 = df_anomalias_final.sort('score_anomalia_total', descending=True).head(5)
    
    for i, row in enumerate(top_5.iter_rows(named=True), 1):
        print(f"\n{i}. TÍTULO: {row['titulo']}")
        print(f"   Score Total: {row['score_anomalia_total']:.1f} - {row['clasificacion_anomalia']}")
        print(f"   Titulares: {row['N_titulares']} | Conflictivos: {row['Diferente_titular']}")
        print(f"   Cambios en períodos: {row['Cambios_en_20_periodos']}")
        
        if row['discrepancias_subterraneas_count'] and row['discrepancias_subterraneas_count'] > 0:
            print(f"   Discrepancias subterráneas: {row['discrepancias_subterraneas_count']} (consistencia: {row['porcentaje_consistencia_sub']:.1f}%)")
        if row['discrepancias_superficiales_count'] and row['discrepancias_superficiales_count'] > 0:
            print(f"   Discrepancias superficiales: {row['discrepancias_superficiales_count']} (consistencia: {row['porcentaje_consistencia_sup']:.1f}%)")
    
    # Ejemplos por tipo de problema
    print(f"\n\n🔍 EJEMPLOS POR TIPO DE PROBLEMA:")
    
    # Títulos con múltiples titulares conflictivos
    print(f"\n1. TÍTULOS CON MÚLTIPLES TITULARES CONFLICTIVOS:")
    titulares_conflictivos = df_anomalias_final.filter(
        (pl.col('N_titulares') > 1) & (pl.col('Diferente_titular') == 0)
    ).sort('N_titulares', descending=True).head(3)
    
    for row in titulares_conflictivos.iter_rows(named=True):
        print(f"   • {row['titulo'][:50]}... | Titulares: {row['N_titulares']} | Score: {row['score_anomalia_total']:.1f}")
    
    # Títulos con peores discrepancias subterráneas
    print(f"\n2. TÍTULOS CON PEORES DISCREPANCIAS SUBTERRÁNEAS:")
    peores_sub = df_anomalias_final.filter(
        pl.col('porcentaje_consistencia_sub').is_not_null()
    ).sort('porcentaje_consistencia_sub').head(3)
    
    for row in peores_sub.iter_rows(named=True):
        print(f"   • {row['titulo'][:50]}... | Consistencia: {row['porcentaje_consistencia_sub']:.1f}% | Score: {row['score_anomalia_total']:.1f}")
    
    # Títulos con peores discrepancias superficiales
    print(f"\n3. TÍTULOS CON PEORES DISCREPANCIAS SUPERFICIALES:")
    peores_sup = df_anomalias_final.filter(
        pl.col('porcentaje_consistencia_sup').is_not_null()
    ).sort('porcentaje_consistencia_sup').head(3)
    
    for row in peores_sup.iter_rows(named=True):
        print(f"   • {row['titulo'][:50]}... | Consistencia: {row['porcentaje_consistencia_sup']:.1f}% | Score: {row['score_anomalia_total']:.1f}")
    
    # Títulos con más cambios de volumen
    print(f"\n4. TÍTULOS CON MÁS CAMBIOS DE VOLUMEN:")
    cambios_volumen = df_anomalias_final.filter(
        pl.col('score_volumenes') > 0
    ).sort(['Anomalia_V_Total', 'Anomalia_V_Subterraneo', 'Anomalia_V_Superficial'], descending=True).head(3)
    
    for row in cambios_volumen.iter_rows(named=True):
        print(f"   • {row['titulo'][:50]}... | Vol.Total: {row['Anomalia_V_Total']}, Sub: {row['Anomalia_V_Subterraneo']}, Sup: {row['Anomalia_V_Superficial']} | Score: {row['score_anomalia_total']:.1f}")

# Ejecutar análisis de ejemplos
mostrar_ejemplos_problematicos()

print(f"\n\n✅ ANÁLISIS COMPLETO FINALIZADO")
print(f"📁 Archivos exportados en el directorio actual:")
print(f"   • df_anomalias_completo.csv")
print(f"   • titulos_prioritarios.csv") 
print(f"   • top_100_anomalias.csv")
print(f"   • resumen_anomalias.csv")
print("="*100)

                        EJEMPLOS DE TÍTULOS PROBLEMÁTICOS

🚨 TOP 5 TÍTULOS CON MAYOR SCORE DE ANOMALÍA:

1. TÍTULO: 04PUE103357/18ASGE00
   Score Total: 23.0 - CRÍTICA
   Titulares: 1 | Conflictivos: None
   Cambios en períodos: 20
   Discrepancias subterráneas: 40 (consistencia: 0.0%)
   Discrepancias superficiales: 40 (consistencia: 0.0%)

2. TÍTULO: 6CHS100642/30APGE94
   Score Total: 21.0 - CRÍTICA
   Titulares: 2 | Conflictivos: 0
   Cambios en períodos: 20
   Discrepancias subterráneas: 60 (consistencia: 0.0%)

3. TÍTULO: 11CHS103721/30HMOC07
   Score Total: 21.0 - CRÍTICA
   Titulares: 2 | Conflictivos: 0
   Cambios en períodos: 20
   Discrepancias subterráneas: 40 (consistencia: 0.0%)

4. TÍTULO: 08GUA120364/12AMDL10
   Score Total: 21.0 - CRÍTICA
   Titulares: 2 | Conflictivos: 0
   Cambios en períodos: 20
   Discrepancias subterráneas: 56 (consistencia: 0.0%)

5. TÍTULO: 12YUC110233/32IMGR00
   Score Total: 21.0 - CRÍTICA
   Titulares: 2 | Conflictivos: 0
   Cambios en períod